## 테마주 분석(ft. 전기자동차)

* 팀장님이 우리 회사와 유사한 테마로 사업을 하고 있는 종목을 수집해서 분석해 달라는 미션을 주셨습니다.
* 코스피에서 해당 테마의 시가총액이 가장 많은 종목을 찾아주세요.
* 해당 테마의 거래량이 가장 많은 종목을 찾아주에세요.
* 최근 52주 최고가에서 현재가 기준으로 가장 손실이 크게 난 종목은 어떤 종목일까요?
* 거래대금이 20위 안에 있는 코스피 종목 중에 "동일업종 PER"가 가장 낮은 종목은 무엇일까요?
* 아래 실습결과에 다른 테마를 적용해 보면 같은 방식으로 여러 테마를 분석해 보실 수 있습니다.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# 시각화를 위한 폰트설정
import platform
import matplotlib.pyplot as plt

system_name = platform.system()

if system_name == "Darwin" :
    font_family = "AppleGothic"
elif system_name == "Windows":
    font_family = "Malgun Gothic"
else:
    # Linux
    !apt -qq -y install fonts-nanum > /dev/null
    import matplotlib.font_manager as fm

    fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
    font = fm.FontProperties(fname=fontpath, size=9)
    fm._rebuild()
    font_family = "NanumBarunGothic"
    
# 폰트설정
plt.rc("font", family=font_family)
# 마이너스폰트 설정
plt.rc("axes", unicode_minus=False)
# 그래프 스타일 설정
plt.style.use("ggplot")


# 그래프에 retina display 적용
%config InlineBackend.figure_format = 'retina'

In [ ]:
pd.Series([1, 3, 5, -7, 9]).plot(title="한글폰트 확인")

In [ ]:
df = pd.read_csv("theme_no_227.csv")
df.shape

In [ ]:
df.columns

## 데이터 전처리
###  데이터 타입 변경
* 시가총액 데이터 수치형태로 변경하기

In [ ]:
df["시가총액(억원)"] = df["시가총액"].str.split("억원", expand=True)[0]
df[["시가총액", "시가총액(억원)", '시가총액순위']].head()

## 파생변수 만들기와 데이터 전처리
* 코스피/코스닥 구분 만들기
* 시가총액 순위 분리하기
* 액면가l매매단위
* 52주최고l최저

In [ ]:
df["코스피/코스닥"] = df['시가총액순위'].str.split(" ", expand=True)[0]
df["시가총액순위(전처리)"] = df['시가총액순위'].str.split(" ", expand=True)[1]
df[["시가총액", "시가총액(억원)", '시가총액순위', '코스피/코스닥', '시가총액순위(전처리)']].head()

### 문자열 자르기 str.split()

In [ ]:
df["액면가"] = df['액면가l매매단위'].str.split("  l  ", expand=True)[0]
df["매매단위"] = df['액면가l매매단위'].str.split("  l  ", expand=True)[1]
df["액면가"] = df["액면가"].str.replace("원", "")
df["액면가"] = df["액면가"].str.replace(",", "")

In [ ]:
df["시가총액순위(전처리)"] = df["시가총액순위(전처리)"].str.replace("위", "").astype(int)

In [ ]:
df["52주최고"] = df["52주최고l최저"].str.split("  l  ", expand=True)[0]
df["52주최저"] = df["52주최고l최저"].str.split("  l  ", expand=True)[1]

### 문자열 대체 str.replace()

In [ ]:
df["52주최고(int)"] = df["52주최고"].str.replace(",", "").astype(int)
df["52주최저(int)"] = df["52주최저"].str.replace(",", "").astype(int)

df["52주차액"] = df["52주최고(int)"] - df["52주최저(int)"]
df["52주차액비율"] = df["52주최고(int)"] / df["52주최저(int)"]

df["현재-최고가"] = df["현재가"] - df["52주최고(int)"]
df["현재/최고가비율"] =  df["현재가"] / df["52주최고(int)"] 

### 실습 
* "동일업종 PER 컬럼"에서 "배" 문자를 제거하고 수치 변수로 변환해 주세요.
* 전처리 결괏값은 df["동일업종 PER(배)"]라는 컬럼에 담아주세요

### 필요한 컬럼만 서브셋으로 가져오기

In [ ]:
df.columns

In [ ]:
cols = ['종목명', '현재가','코스피/코스닥', "52주최고(int)", "52주최저(int)", 
    "52주차액", "52주차액비율", "현재-최고가", "현재/최고가비율", "동일업종 PER(배)", "동일업종 등락률", "거래량", "거래대금", "시가총액순위(전처리)"]

In [ ]:
df[cols].sort_values(
    "52주차액비율", ascending=False).head(30)

### 서브셋 만들기
* 서브셋 만들기
* 정렬하기

### 코스피에서 해당 테마의 시가총액이 가장 많은 종목을 찾아주세요.

In [ ]:
코스피 = df[df["코스피/코스닥"] == "코스피"].sort_values("시가총액순위(전처리)")
코스피[cols].head(20)

### 해당 테마의 거래량이 가장 많은 종목을 찾아주세요.

### 코스피 해당 테마 종목 중 최근 52주 최고가에서 현재가 기준으로 가장 손실이 크게 난 종목은 어떤 종목일까요?

In [ ]:
코스피[cols].sort_values("현재-최고가", ascending=True).head(20)

### 코스피 해당 테마에서 거래대금이 20위 안에 있는 코스피 종목 중에 "동일업종 PER"가 가장 낮은 종목은 무엇일까요?

In [ ]:
코스피거래대금 = 코스피[cols].sort_values("거래대금", ascending=False).head(20)
코스피거래대금

In [ ]:
# 동일업종 PER
코스피거래대금.sort_values("동일업종 PER(배)")


## 상관분석

상관 분석(相關 分析, Correlation analysis) 또는 '상관관계' 또는 '상관'은 확률론과 통계학에서 두 변수간에 어떤 선형적 또는 비선형적 관계를 갖고 있는지를 분석하는 방법이다. 두 변수는 서로 독립적인 관계이거나 상관된 관계일 수 있으며 이때 두 변수간의 관계의 강도를 상관관계(Correlation, Correlation coefficient)라 한다. 상관분석에서는 상관관계의 정도를 나타내는 단위로 모상관계수로 ρ를 사용하며 표본 상관 계수로 r 을 사용한다.

상관관계의 정도를 파악하는 상관 계수(相關係數, Correlation coefficient)는 두 변수간의 연관된 정도를 나타낼 뿐 인과관계를 설명하는 것은 아니다. 두 변수간에 원인과 결과의 인과관계가 있는지에 대한 것은 회귀분석을 통해 인과관계의 방향, 정도와 수학적 모델을 확인해 볼 수 있다.

* 출처 : https://ko.wikipedia.org/wiki/%EC%83%81%EA%B4%80_%EB%B6%84%EC%84%9D

### 피어슨 상관계수

* r 값은 X 와 Y 가 완전히 동일하면 +1, 전혀 다르면 0, 반대방향으로 완전히 동일 하면 –1 을 가진다. 
* 결정계수(coefficient of determination)는 r^2 로 계산하며 이것은 X 로부터 Y 를 예측할 수 있는 정도를 의미한다.

```
일반적으로
r이 -1.0과 -0.7 사이이면, 강한 음적 선형관계,
r이 -0.7과 -0.3 사이이면, 뚜렷한 음적 선형관계,
r이 -0.3과 -0.1 사이이면, 약한 음적 선형관계,
r이 -0.1과 +0.1 사이이면, 거의 무시될 수 있는 선형관계,
r이 +0.1과 +0.3 사이이면, 약한 양적 선형관계,
r이 +0.3과 +0.7 사이이면, 뚜렷한 양적 선형관계,
r이 +0.7과 +1.0 사이이면, 강한 양적 선형관계
```

* https://ko.wikipedia.org/wiki/%EC%83%81%EA%B4%80_%EB%B6%84%EC%84%9D#%ED%94%BC%EC%96%B4%EC%8A%A8_%EC%83%81%EA%B4%80_%EA%B3%84%EC%88%98



### 상관계수 구하기

In [ ]:
kospi_corr = 코스피.corr().dropna(how="all", axis=0).dropna(how="all", axis=1)
kospi_corr

## 상관계수

In [ ]:
import seaborn as sns

In [ ]:
mask = np.triu(np.ones_like(kospi_corr, dtype=bool))
plt.figure(figsize=(20, 6))
sns.heatmap(kospi_corr, cmap="coolwarm", annot=True, mask=mask)

### 종목별 상관계수

In [ ]:
num_desc = 코스피.set_index("종목명").describe()
num_cols = num_desc.columns.tolist()
num_cols.append("종목명")

In [ ]:
item_corr = 코스피[num_cols].set_index("종목명").T.corr()

In [ ]:
item_corr["삼성SDI"].sort_values(ascending=False)

* 실습결과에 다른 테마를 적용해 보면 같은 방식으로 여러 테마를 분석해 보실 수 있습니다.